DE Bay Breeze Detection
Baby Code
LWSD1 - NDBC Station LWSD1 - 8557380 - Lewes, DE


In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
import glob

In [2]:
# Set the directory containing the buoy data files
data_directory = 'G:/My Drive/Summer Research/BB detection/NDBC_Buoys'

In [3]:
# Set the specific file name
file_path = os.path.join(data_directory, 'lwsd1h2021.txt')

In [5]:
# Read the weather data from the text file
data = pd.DataFrame(pd.read_csv(file_path, delimiter='\s+', skiprows=[1],
                   usecols=['#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'PRES',  'ATMP',  'WTMP' ],
                   na_filter=True, 
                   na_values=['99.0', '99.00', '999.0']))


# Rename the '#YY', 'MM' , 'DD', 'hh', 'mm' columns
data.rename(columns={'#YY':'Year', 'MM':'Month', 'DD':'Day', 'hh':'Hour', 'mm':'Minute'}, inplace=True)

In [6]:
# Concatenate the Year, Month, Day, Hour, Minute in one column and set this column as Index

data['DateTime'] = pd.to_datetime(data[['Year', 'Month', 'Day', 'Hour', 'Minute']])

data.index = data.DateTime

In [7]:
# Criteria for bay breeze detection

temperature_threshold = 0.5  # Celsius Degree
wind_speed_threshold = 2  # m/s
direction_change_threshold = 45  # Degree

In [9]:
# Filter the data for the desired date range
start_date = pd.to_datetime('2021-5-1')
end_date = pd.to_datetime('2021-9-30')
data_range = data[(data['DateTime'] >= start_date) & (data['DateTime'] <= end_date)]

# Filter the data for the desired time range (12 PM to 3 AM next day)
filtered_data = data_range[
    ((data_range['Hour'] >= 12) & (data_range['Hour'] <= 23)) #|
    #((data_range['Hour'] >= 0) & (data_range['Hour'] <= 3))
]

In [43]:
bay_breeze_passages = []  # List to store the detected sea breeze passages

# Convert 'DateTime' column to datetime format
filtered_data['DateTime'] = pd.to_datetime(filtered_data['DateTime'])

# Create data frames to store passage information
passages_df = pd.DataFrame(columns=['DateTime', 'ATMP', 'WDIR'])

# Iterate through the data to detect sea breeze passages
for i in range(len(filtered_data) - 5):
    curr_row = filtered_data.iloc[i]
    next_row = filtered_data.iloc[i+5]
    
    criteria = 0
    # Check if the temperature exceeds the threshold
    if curr_row['ATMP'] - next_row['ATMP'] > temperature_threshold:
        criteria += 1
    # Calculate the change in wind speed over 0.5 hours
    speed_change = next_row['WSPD'] - curr_row['WSPD']
    
    if curr_row['WSPD'] <= 10 and next_row['WSPD'] <= 10:
        if speed_change > wind_speed_threshold:
            criteria += 1
    # Calculate the change in wind direction over 0.5 hours
    direction_change = curr_row['WDIR'] - next_row['WDIR']  # Assuming 5 minutes interval
    
    # Check if the wind direction change exceeds the threshold
    if abs(direction_change) > direction_change_threshold and 0 < curr_row['WDIR'] <= 180:
        criteria += 1
    
    if criteria >= 3:
        passage_time = curr_row['DateTime']  # Time of sea breeze passage
        temperature = curr_row['ATMP']  # Air temperature at the passage time
        wind_direction = curr_row['WDIR']  # Wind direction at the passage time
        wind_speed = curr_row['WSPD'] # Wind speed at the passage time
            
        # Append the passage information to the list
        bay_breeze_passages.append((passage_time, temperature, wind_direction))
                
        # Append the passage information to the data frame
        passages_df = passages_df.append({'DateTime': passage_time, 'ATMP': temperature, 'WDIR': wind_direction, 'WSPD': wind_speed}, ignore_index=True)
print(passages_df)


C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\802500066.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['DateTime'] = pd.to_datetime(filtered_data['DateTime'])
C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\802500066.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  passages_df = passages_df.append({'DateTime': passage_time, 'ATMP': temperature, 'WDIR': wind_direction, 'WSPD': wind_speed}, ignore_index=True)
C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\802500066.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  passages_df = passages_df

               DateTime  ATMP   WDIR  WSPD
0   2021-05-04 22:30:00  26.3   85.0   0.0
1   2021-05-04 22:36:00  25.9  115.0   0.6
2   2021-05-07 16:00:00  13.6   59.0   1.8
3   2021-05-08 20:18:00  12.3    9.0   4.3
4   2021-05-11 23:30:00  16.1  154.0   0.5
5   2021-05-11 23:36:00  16.0  175.0   1.4
6   2021-05-11 23:42:00  15.9  165.0   1.7
7   2021-05-13 22:00:00  18.5   27.0   0.7
8   2021-05-17 18:18:00  19.6   93.0   2.4
9   2021-05-18 23:30:00  20.6  177.0   2.7
10  2021-05-18 23:36:00  20.6  177.0   2.3
11  2021-05-18 23:42:00  20.4  168.0   2.9
12  2021-05-18 23:48:00  20.3  174.0   2.7
13  2021-05-26 23:42:00  27.3  179.0   1.4
14  2021-05-27 23:30:00  24.2  145.0   1.7
15  2021-05-27 23:36:00  24.0  170.0   2.7
16  2021-05-27 23:42:00  24.0  173.0   2.8
17  2021-05-27 23:48:00  23.9  173.0   3.1
18  2021-05-27 23:54:00  23.5  174.0   3.6
19  2021-06-14 23:30:00  25.4  137.0   1.1
20  2021-06-22 13:54:00  25.5   59.0   0.9
21  2021-06-22 14:00:00  25.4   22.0   1.4
22  2021-06

C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\802500066.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  passages_df = passages_df.append({'DateTime': passage_time, 'ATMP': temperature, 'WDIR': wind_direction, 'WSPD': wind_speed}, ignore_index=True)


In [44]:
# Convert 'DateTime' column to datetime format
passages_df['DateTime'] = pd.to_datetime(passages_df['DateTime'])

# Create a dataframe to store the first passage after 12pm for each day
first_passage_df = pd.DataFrame(columns=['Date', 'FirstPassageTime', 'ATMP', 'WDIR', 'WSPD'])

# Iterate through the detected sea breeze passages to find the first passage after 12pm for each day
for passage_date in passages_df['DateTime'].dt.date.unique():
    passages_date_filtered = passages_df[passages_df['DateTime'].dt.date == passage_date]
    
    # Filter out passages before 12 pm
    passages_date_filtered = passages_date_filtered[passages_date_filtered['DateTime'].dt.hour >= 12]
    
    if not passages_date_filtered.empty:
        first_passage = passages_date_filtered.iloc[0]
        
        # Append the first passage information to the dataframe
        first_passage_df = first_passage_df.append({'Date': passage_date, 'FirstPassageTime': first_passage['DateTime'], 
                                                    'ATMP': first_passage['ATMP'], 
                                                    'WDIR': first_passage['WDIR'],
                                                    'WSPD': first_passage['WSPD']}, 
                                                   ignore_index=True)
print(first_passage_df)

C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\2740400230.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_passage_df = first_passage_df.append({'Date': passage_date, 'FirstPassageTime': first_passage['DateTime'],
C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\2740400230.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_passage_df = first_passage_df.append({'Date': passage_date, 'FirstPassageTime': first_passage['DateTime'],
C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\2740400230.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_passage_df = first_passage_df.append({'Date': passage_date, 'FirstPassageTime': first_passage['DateTime'],
C:\Users\leuyy\AppData\Local\Temp\ipykernel_11092\27

          Date     FirstPassageTime  ATMP   WDIR WSPD
0   2021-05-04  2021-05-04 22:30:00  26.3   85.0  0.0
1   2021-05-07  2021-05-07 16:00:00  13.6   59.0  1.8
2   2021-05-08  2021-05-08 20:18:00  12.3    9.0  4.3
3   2021-05-11  2021-05-11 23:30:00  16.1  154.0  0.5
4   2021-05-13  2021-05-13 22:00:00  18.5   27.0  0.7
5   2021-05-17  2021-05-17 18:18:00  19.6   93.0  2.4
6   2021-05-18  2021-05-18 23:30:00  20.6  177.0  2.7
7   2021-05-26  2021-05-26 23:42:00  27.3  179.0  1.4
8   2021-05-27  2021-05-27 23:30:00  24.2  145.0  1.7
9   2021-06-14  2021-06-14 23:30:00  25.4  137.0  1.1
10  2021-06-22  2021-06-22 13:54:00  25.5   59.0  0.9
11  2021-06-23  2021-06-23 23:36:00  20.3  148.0  2.2
12  2021-06-26  2021-06-26 15:36:00  26.3  130.0  4.3
13  2021-07-02  2021-07-02 23:30:00  23.2  169.0  1.9
14  2021-07-08  2021-07-08 23:30:00  24.4  130.0  0.7
15  2021-07-15  2021-07-15 23:30:00  28.4  159.0  2.0
16  2021-07-20  2021-07-20 23:48:00  28.1  173.0  1.9
17  2021-07-21  2021-07-21 2

In [181]:
# Save the first passages as an EXCEL file
first_passage_df.to_excel('first_passages_2012.xlsx', index=False)

In [42]:
# Save the first passages as a CSV file
first_passage_df.to_csv('first_passages_critno.csv', index=False)

In [1]:
# Save the first passages as a text file
first_passage_df.to_csv('first_passages_2021.txt', sep='\t', index=False)

NameError: name 'first_passage_df' is not defined